In [23]:
##import libs
import numpy as np
import plotly
from plotly.subplots import make_subplots
import os
import matplotlib.pyplot as plt #to enable plotting within notebook
import matplotlib
from nilearn import image as nimg
from nilearn import plotting 
from bids.layout import BIDSLayout
import bids
from matplotlib.pyplot import figure
import mpld3
import pandas as pd
from pathlib import Path   
import nibabel as nb 
import plotly.express as px
from nilearn.datasets import fetch_icbm152_brain_gm_mask
from nilearn.plotting import plot_roi
from nilearn.glm.first_level import make_first_level_design_matrix, run_glm
import seaborn as sns
from scipy.stats import norm
from nilearn.glm.contrasts import compute_contrast
from nilearn.plotting import plot_contrast_matrix
from nilearn.reporting import make_glm_report
from nilearn.interfaces.bids import save_glm_to_bids
from ordered_set import OrderedSet
from nilearn.glm.first_level import first_level_from_bids
import pickle
import os.path
import pathlib
import gc
from nilearn import image
from templateflow import api as tflow
from nilearn.maskers import NiftiMasker
import traceback
import logging
from nilearn.glm.first_level import FirstLevelModel
import nilearn as nl
from nilearn.interfaces.fmriprep import load_confounds
import json
from nilearn import surface
import cv2

import nibabel as nb
from pathlib import Path              # Combine path elements with /
from pprint import pprint             # Pretty-printing
import numpy as np                    # Numeric Python
from matplotlib import pyplot as plt  # Matlab-ish plotting commands
from nilearn import plotting as nlp   # Nice neuroimage plotting
import transforms3d                   # Work with affine algebra
from scipy import ndimage as ndi      # Operate on N-dimensional images
import nibabel.testing                # For fetching test data
import hcp_utils as hcp
##import libs for glm and plots
import nilearn
import warnings
from nilearn.connectome import ConnectivityMeasure
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline


In [24]:
##Set path to the data folder
os.chdir('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/')
print("The working directory has been changed!")


The working directory has been changed!


In [25]:
## set path & get layout object for later use:
data_dir = '/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/'


In [26]:
## set variable for each dir task = WM

# task_label = ['SLD', 'SSI', 'SSD', 'SLI', 'VLD', 'VSI', 'VSD', 'VLI']
space_label = "MNI152NLin2009cAsym"
derivatives_folder = "derivatives/fmriprep"
# direction = ['LR', 'RL']
#contrast_type = 'story-math'
ses = 'T1'

In [27]:


subject_ID_df  = pd.read_csv(data_dir + "valid_subj_list.csv", dtype=str, index_col=0)
subject_ID = list(subject_ID_df.iloc[:,0])
subject_ID = [str(x[4:]) for x in subject_ID]



In [28]:
len(subject_ID)

79

In [29]:
## function: extract desired confounds
def manage_confounds(original_confs):


       cosine = original_confs.filter(regex='^cosine',axis=1)
       cosine_regs = list(cosine.columns)
       non_steady = original_confs.filter(regex='^non_steady_state',axis=1)
       a_comp_cor = ['c_comp_cor_00', 'c_comp_cor_01', 'c_comp_cor_02', 'c_comp_cor_03', 'c_comp_cor_04',
                      'w_comp_cor_00', 'w_comp_cor_01', 'w_comp_cor_02', 'w_comp_cor_03', 'w_comp_cor_04']
       movement_regs = ['rot_x', 'rot_x_derivative1','rot_y','rot_y_derivative1','rot_z','rot_z_derivative1',
                      'trans_x', 'trans_x_derivative1','trans_y', 'trans_y_derivative1','trans_z', 'trans_z_derivative1']
       fd = ['framewise_displacement', 'std_dvars']
       desired_confs = a_comp_cor + cosine_regs + movement_regs + fd


       if ~set(desired_confs).issubset(original_confs.columns):
              all_confs = original_confs.columns.values.tolist()
              mising_confs = list(set(desired_confs) - set(all_confs))
              final_confounds = list(OrderedSet(desired_confs) - OrderedSet(mising_confs))
       else:
              final_confounds = desired_confs
       
       #get dummy scan rows     
       temp = original_confs.isna().any(axis=1)
       na_ind = list(temp[temp].index)
       dummy = list(np.where(non_steady == 1)[0])
       dummy_scans = list(set(na_ind+dummy))
       confs_final = original_confs.drop(dummy_scans,axis=0,inplace=False)
       confs_final = confs_final.reset_index(drop=True)

       confs_final = confs_final.loc[:,final_confounds]
       confs_final.insert(confs_final.shape[1],'linear_trend', range(len(dummy_scans), original_confs.shape[0]))

       return confs_final, dummy_scans


In [30]:
## function: create & save plots
def manage_plots(unparced_z, unparced_eff , sub_id, contrast_t, task_label, output_dir):
     surface_zs = nl.plotting.view_surf(hcp.mesh.inflated,
     hcp.cortex_data(unparced_z), 
     bg_map=hcp.mesh.sulc, cmap='bwr')
     surface_zs.save_as_html(output_dir + '/sub-%s_ses-%s_task-%s_contrast-%s_z_parcelated_plot.html' %(sub_id, ses, task_label, contrast_t))
     surface_eff = nl.plotting.view_surf(hcp.mesh.inflated,
     hcp.cortex_data(unparced_eff), 
     bg_map=hcp.mesh.sulc, cmap='bwr')
     surface_eff.save_as_html(output_dir + '/sub-%s_ses-%s_task-%s_contrast-%s_effect_parcelated_plot.html' %(sub_id, ses, task_label, contrast_t))
     plt.close('all')
     gc.collect()



In [31]:
## function: prepare events file:
def prepare_events(ind_events,dummy_scans, t_r):
    n_dummy = len(dummy_scans)
    if n_dummy != 0:
        #print(n_dummy)
        new_zero = n_dummy * t_r
        ind_events['onset'] = ind_events['onset'] - new_zero
        invalid_row_ind = np.asarray(ind_events['onset'] < 0).nonzero()
        if len(invalid_row_ind[0]) != 0:
            del_row = []
            for i in invalid_row_ind[0]:
                # print(i)
                ii=i
                new_dur = ind_events.at[ii,'onset']+ind_events.at[ii,'duration']
                if new_dur > 0:
                    ind_events.at[ii,'onset'] = 0 
                    ind_events.at[ii,'duration'] = new_dur
                else:
                    del_row.append(ii)
            ind_events = ind_events.drop(del_row)  
    
    return  ind_events


In [32]:
def to_block(ind_events):

    ind_events = ind_events[['onset', 'duration', 'trial_type']]
    for i in range(len(ind_events) - 2):
        if ind_events.at[i, 'trial_type'] == 'no_response':
            ind_events.at[i, 'trial_type'] = ind_events.at[i + 3, 'trial_type']
    # print(ind_events)
    # Identify block starts and ends
    ind_events['next_trial_type'] = ind_events['trial_type'].shift(-1)
    ind_events['block_end'] = ind_events['trial_type'] != ind_events['next_trial_type']
    ind_events['block_start'] = ind_events['block_end'].shift(1, fill_value=True)

    # Compute block onsets and ends
    block_starts = ind_events[ind_events['block_start']].copy()
    block_ends = ind_events[ind_events['block_end']].copy()

    # Ensure the same index for merging
    block_starts.index = range(len(block_starts))
    block_ends.index = range(len(block_ends))

    # Merge to get block start and end times
    blocks = block_starts.copy()
    blocks['block_onset'] = block_starts['onset']
    blocks['block_end'] = block_ends['onset'] + block_ends['duration']

    # Compute the block duration
    blocks['block_duration'] = blocks['block_end'] - blocks['block_onset']

    # Select relevant columns
    block_df = blocks[['block_onset', 'block_duration', 'trial_type']]
    block_df.columns = ['onset', 'duration', 'trial_type']

    return block_df





In [33]:
# function: Reshape, Parcellate 
def R_P (stat_map):
    ##parcelate full
    col = np.ones(len(stat_map))
    map_2d=np.stack((stat_map,col),axis=1)
    map_parc_data = np.transpose(map_2d)

    parced_map = hcp.parcellate(map_parc_data, hcp.mmp)
    unparced_map = hcp.unparcellate(parced_map[0], hcp.mmp)

    return parced_map, unparced_map


In [34]:
# function:  convert to cifti 
def convert_to_cifti (stat_map, axes, t_r):
    ##prepare save as cifti
    #map = np.reshape(stat_map, (-1, stat_map.shape[0]))
    map = stat_map
    #save contrasts as cifti full
    scalar_axis_map = nb.cifti2.SeriesAxis(start=0, step=t_r, size=map.shape[0])  # Takes a list of names, one per row
    map_header = nb.Cifti2Header.from_axes([scalar_axis_map, axes[1]])
    map_img = nb.Cifti2Image(map, header = map_header, ) 
                
    return map_img


In [35]:
## function: flag outliers:
def flag_outliers(confounds):
    fd_threshold = 0.5  #from xcp-d #.3 in thomas paper
    std_DVARS_threshold = 1.5  #from nilearn load_conf doc
    #DVARS_threshold = 50 #from thomas paper
    outlier_index = []  # just vols with high fd
    censor_index = []  #all vols to be removed including high motion and surroundings
    for ind , vol in enumerate(confounds['framewise_displacement']):
        if vol > fd_threshold or confounds['std_dvars'][ind] > std_DVARS_threshold: 
            outlier_index.append(ind)
            censor_index.extend([ind-2, ind-1, ind, ind+1])
    for item in outlier_index:
        if item+6 in censor_index:
            censor_index.extend(list(range(item+2 , item+6)))
            
    censor_index = [x for x in censor_index if x > -1 and x < confounds.shape[0]] 
    censor_set = set(censor_index)
    censor_index = list(censor_set)
    censor_index.sort()
    data_ind = list(confounds.index)
    censored = list(OrderedSet(data_ind) - OrderedSet(censor_index))
    censored_mask = np.asarray(censored)  # contains vols left after removing high motion and the surroundings
    bi_mask = list(OrderedSet(data_ind) - OrderedSet(outlier_index))
    bi_mask = [False if s in outlier_index else True for s in data_ind]
    bi_mask = np.asarray(bi_mask) 

    return outlier_index, censor_index, censored_mask, bi_mask


In [36]:
def compute_conn(parced_sig, path, sub_label, task_label , cen):
    headers = list(hcp.mmp.labels.values())[1:]
    ##plot nilearn parced correlation matrix
    correlation_measure = ConnectivityMeasure(kind="correlation")
    correlation_matrix = correlation_measure.fit_transform([parced_sig])[0]

    pd.DataFrame(correlation_matrix, columns = headers).to_csv(path +
                                    '/sub-%s_ses-%s_task-%s_space-fsLR_atlas-Glasser_desc-%s_measure-pearsoncorrelation_conmat.tsv' %(sub_label, ses, task_label, cen), sep='\t')
    # plot connectivity matrix
    conmat_fig = plotting.plot_matrix(
        correlation_matrix,
        figure=(10, 8),
        labels=headers, 
        vmax=1,
        vmin=-1,
        #title="Confounds",
        #reorder=True,
    )
    conmat_fig.figure.savefig(path + '/sub-%s_ses-%s_task-%s_space-fsLR_atlas-Glasser_desc-%s_measure-pearsoncorrelation_conmat.png' %(sub_label, ses, task_label, cen))
    gc.collect()


In [37]:
def despiking(signal):
    cen_desp = signal.copy()
    spike_threshold = 3.0
    for i in range(cen_desp.shape[1]):  # loop over columns
        vertex_data = cen_desp[:, i]
        mean_value = np.mean(vertex_data)
        spike_indices = np.where(np.abs(vertex_data - mean_value) > spike_threshold)[0]
        
        # Calculate mean without spike values
        non_spike_indices = np.setdiff1d(np.arange(len(vertex_data)), spike_indices)
        if len(non_spike_indices) > 0:
            replacement_value = np.mean(vertex_data[non_spike_indices])
        else:
            replacement_value = mean_value  # fallback if all are spikes
        
        vertex_data[spike_indices] = replacement_value
        cen_desp[:, i] = vertex_data  # reassign modified column

    return cen_desp


In [38]:
## function: run GLM and extract contrasts
def surf_contrast(iter, sub_label, data_dir, output_dir):
    try:
        warnings.filterwarnings(action='ignore')
        # map_dict = {1: "oneback", 2:"twoback"}
        path = (output_dir+'sub-{}'.format(sub_label))
        #print(path)
        if not os.path.exists(path):
            os.mkdir(path)
        #plt.ioff()

        # set up lists for averaging across direction
        cen_res_list = []
        uncen_res_list = []
        censored_vols = []
        outlier_vols = []
        half_vols = []
        print('start:'+sub_label)

        task_labels = ['VLD', 'VSI', 'VSD', 'VLI']
        # task_labels = ['SLD', 'SSI', 'SSD', 'SLI']
        for task_label in task_labels:

            json_file = data_dir + 'derivatives/fmriprep/sub-%s/ses-T1/func/sub-%s_ses-T1_task-%s_space-fsLR_den-91k_bold.json' %(sub_label,sub_label, task_label)
            if os.path.isfile(json_file):
                with open(json_file, 'r') as f:
                    t_r = json.load(f)['RepetitionTime']

                fmriprep_bold = (data_dir + 'derivatives/fmriprep/sub-%s/ses-T1/func/sub-%s_ses-T1_task-%s_space-fsLR_den-91k_bold.dtseries.nii' %(sub_label, sub_label, task_label))
                eve_dir = (data_dir + 'sub-%s/ses-T1/func/sub-%s_ses-T1_task-%s_events.tsv' %(sub_label, sub_label, task_label))
                conf_dir = (data_dir + 'derivatives/fmriprep/sub-%s/ses-T1/func/sub-%s_ses-T1_task-%s_desc-confounds_timeseries.tsv' %(sub_label, sub_label, task_label))

                events = pd.read_csv(eve_dir , sep='\t')
                confounds = pd.read_csv(conf_dir, sep='\t')
                #func_mni_img = nimg.load_img(fmriprep_bold)

                # call confounds function:
                [confounds_final, dummy_scans] = manage_confounds(confounds)

                # confounds_final = confounds_final.reset_index(drop=True)

                # calculate frametimes using confounds file n_rows and T_R
                frame_times = (np.arange(confounds_final.shape[0]) * t_r)# + (t_r/2)

                # call prepare event file function
                ind_events = events.copy()
                # fix events names
                ind_events = ind_events.replace(regex=['1-'],value='one')
                ind_events = ind_events.replace(regex=['2-'],value='two')
                # ind_events['trial_type']= ind_events['trial_type'].map(map_dict)
                # print(ind_events.loc[:,'trial_type'])
                ind_events = to_block(ind_events)

                ind_events = prepare_events(ind_events, dummy_scans, t_r)
                # print(ind_events.loc[:,'trial_type'])
                # ind_events = ind_events.reset_index(drop=True)

                # save events file
                ind_events.to_csv(output_dir + 
                                'sub-%s/sub-%s_ses-%s_task-%s_events.tsv' %(sub_label, sub_label, ses, task_label), sep='\t')
                confounds_final.to_csv(output_dir + 
                                'sub-%s/sub-%s_ses-%s_task-%s_confs.tsv' %(sub_label, sub_label, ses, task_label), sep='\t')
                
                # identify and extract outliers
                outliers, censor_ind, censored_mask, bi_mask = flag_outliers(confounds_final)
                # print(censor_ind)
                censored_vols.append(len(censor_ind))
                outlier_vols.append(len(outliers))
                half_vols.append(confounds_final.shape[0]/2)
                pd.DataFrame(bi_mask).to_csv(path +
                                                '/sub-%s_ses-%s_task-%s_outliers.tsv' %(sub_label, ses, task_label), sep='\t')
                
                confounds_final = confounds_final.drop(['framewise_displacement', 'std_dvars'],axis=1,inplace=False) # drop fd and dvars cols before creating design mat

                # create design mat
                design_matrix = make_first_level_design_matrix(
                                                                frame_times, events=ind_events,
                                                                hrf_model='spm', drift_model= None,
                                                                add_regs=confounds_final)

                # for test :
                unt_des_mat = design_matrix.copy()
                # remove all flagged vols/ motion outliers from design mat for censored
                cenlist = []
                desmat_ind = design_matrix.index.tolist()
                for i , ind in enumerate(desmat_ind):
                    if i in censor_ind:
                        cenlist.append(ind)
                cen_des_mat = design_matrix.drop(cenlist,axis=0,inplace=False)

                # add outliers to design mat for uncensored
                for n , ind in enumerate(outliers):
                    col_name = 'outlier%s' %str(n)
                    out_vals = np.zeros(len(confounds_final))
                    out_vals[ind] = 1
                    design_matrix.insert(design_matrix.shape[1], col_name , out_vals)

                design_plot = plotting.plot_design_matrix(design_matrix)
                design_plot.figure.savefig(path + '/sub-%s_ses-%s_task-%s_design.svg' %(sub_label, ses, task_label))
                design_plot2 = plotting.plot_design_matrix(cen_des_mat)
                design_plot2.figure.savefig(path + '/sub-%s_ses-%s_task-%s_censored-design.svg' %(sub_label, ses, task_label))
                cen_des_mat.to_csv(path + '/sub-%s_ses-%s_task-%s_censored-design.tsv' %(sub_label, ses, task_label), sep='\t')
                design_matrix.to_csv(path + '/sub-%s_ses-%s_task-%s_design.tsv' %(sub_label, ses, task_label), sep='\t')

                del confounds, events

                # load image file and remove dummy scans:
                cifti = nb.load(fmriprep_bold)
                cifti_data = cifti.get_fdata()
                cifti_hdr = cifti.header
                nifti_hdr = cifti.nifti_header
                axes = [cifti_hdr.get_axis(i) for i in range(cifti.ndim)]
                cifti_data = cifti_data[len(dummy_scans):,:]     #uncensored
                #standardize:
                std_uncen = nl.signal.clean(cifti_data, detrend=False, standardize='zscore_sample', confounds=None, standardize_confounds=False,
                    filter=False, low_pass=None, high_pass=None, t_r=t_r, ensure_finite=False)
                #despike:
                std_desp = despiking(std_uncen)
                #scrub:
                std_desp_cen = np.delete(std_desp, censor_ind, axis=0)   #censored - despiked

                del cifti
                
                # fit the model 
                labels, estimates = run_glm(std_desp_cen, cen_des_mat.values, noise_model='ols')  # to censored signal
                res_sig_cen = estimates[0.0].residuals

                labels2, estimates2 = run_glm(std_desp, design_matrix.values, noise_model='ols')  # to uncensored signal
                res_sig_uncen = estimates2[0.0].residuals
                fig1, ax1 = plt.subplots()

                # Plot the data on the axis

                # ax1.plot(cifti_data[:, 4], color='yellow')
                ax1.plot(std_uncen[:, 4], color='purple')
                ax1.plot(std_desp[:, 4], color='black')
                ax1.plot(std_desp_cen[:, 4], color='pink')

                # Save the figure
                fig1.savefig(path + '/timeseries.png')
                fig2, ax2 = plt.subplots()
                ax2.plot(res_sig_uncen[:, 4], color='red')
                # ax2.plot(res_sig_cen[:, 4], color='blue')
                # ax2.plot(interplotated_res[:, 4], color='green')
                # ax2.plot(filtered_interplotated_res[:, 0], color='orange')  # Uncomment if needed
                # Save the figure
                fig2.savefig(path + '/res_timeseries.png')
                # labels3, estimates3 = run_glm(cen_desp_cifti_data, cen_des_mat.values, noise_model='ols')  # to uncensored signal
                # res_sig_unt = estimates3[0.0].residuals

                del cifti_data, design_matrix, confounds_final, std_uncen, std_desp, std_desp_cen

                # save residuals
                uncen_res_img = convert_to_cifti(res_sig_uncen, axes, t_r)
                uncen_res_img.to_filename(path + '/sub-%s_ses-%s_task-%s_space-fsLR_den-91k_desc-uncensoredDespiked_residuals.dtseries.nii' %(sub_label, ses, task_label))
                cen_res_img = convert_to_cifti(res_sig_cen, axes, t_r)
                cen_res_img.to_filename(path + '/sub-%s_ses-%s_task-%s_space-fsLR_den-91k_desc-censoredDespiked_residuals.dtseries.nii' %(sub_label, ses, task_label))

                # interpolate uncensored residual
                interplotated_res = res_sig_uncen
                remained_vol = frame_times[bi_mask]
                remained_x = interplotated_res[bi_mask, :]
                cubic_spline_fitter = CubicSpline(remained_vol, remained_x)
                volumes_interpolated = cubic_spline_fitter(frame_times)
                interplotated_res[~bi_mask, :] = volumes_interpolated[~bi_mask, :]
                # inter_res_img = convert_to_cifti(interplotated_res, axes, t_r)
                # inter_res_img.to_filename(path + '/sub-%s_ses-%s_task-%s_dir-%s_space-fsLR_den-91k_desc-interpolatedDenoised_residuals.dtseries.nii' %(sub_label, ses, task_label, dir[dir_ind]))

                # perform filtering
                # sr = 1.0 / t_r
                #filtered_interplotated_res = nl.signal.butterworth(interplotated_res, sr , low_pass=0.08, high_pass=0.01, order=2)

                # parcellate:
                parced_cen_res = hcp.parcellate(res_sig_cen, hcp.mmp)
                parced_uncen_res = hcp.parcellate(res_sig_uncen, hcp.mmp)
                parced_interplotated_res = hcp.parcellate(interplotated_res, hcp.mmp)
                ## read glasser header from hcp
                headers = list(hcp.mmp.labels.values())[1:]
                # save parcellation
                # pres_img = convert_to_cifti(parced_res, axes, t_r)
                # pres_img.to_filename(path + '/sub-%s_task-%s_dir-%s_space-fsLR_den-91k_desc-uncensoredDenoisedParcelated_residuals.dtseries.nii' %(sub_label, task_label, dir[dir_ind]))
                pd.DataFrame(parced_uncen_res, columns = headers).to_csv(path +
                                    '/sub-%s_ses-%s_task-%s_space-fsLR_atlas-Glasser_desc-uncensoredDespiked_parcelations.tsv' %(sub_label, ses, task_label), sep='\t')
                pd.DataFrame(parced_cen_res, columns = headers).to_csv(path +
                                    '/sub-%s_ses-%s_task-%s_space-fsLR_atlas-Glasser_desc-censoredDespiked_parcelations.tsv' %(sub_label, ses, task_label), sep='\t')
                pd.DataFrame(parced_interplotated_res, columns = headers).to_csv(path +
                                    '/sub-%s_ses-%s_task-%s_space-fsLR_atlas-Glasser_desc-censoredDespikedInterp_parcelations.tsv' %(sub_label, ses, task_label), sep='\t')
                # connectivity:
                compute_conn(parced_cen_res, path, sub_label, task_label, "censored")
                compute_conn(parced_uncen_res, path, sub_label, task_label , "uncensored")
                compute_conn(parced_interplotated_res, path, sub_label, task_label , "censoredInterp")
                # keep for sum of runs
                cen_res_list.append(res_sig_cen)
                uncen_res_list.append(res_sig_uncen)
                del  cen_res_img, parced_cen_res , res_sig_uncen , uncen_res_img , res_sig_cen, parced_uncen_res
                plt.close('all')

        plt.close('all')

        dirM = 'verbal' # verbalspatial
        # # combine directions and calculate mean 
        full_cen_res = np.concatenate(cen_res_list, axis=0)
        parced_full_cen_res = hcp.parcellate(full_cen_res, hcp.mmp)
        pd.DataFrame(parced_full_cen_res, columns = headers).to_csv(path +
                                            '/sub-%s_task-%s_space-fsLR_atlas-Glasser_desc-censoredDespiked_parcelations.tsv' %(sub_label, dirM), sep='\t')
        compute_conn(parced_full_cen_res, path, sub_label, dirM, "censored")
        
        full_uncen_res = np.concatenate(uncen_res_list, axis=0)
        parced_full_uncen_res = hcp.parcellate(full_uncen_res, hcp.mmp)
        pd.DataFrame(parced_full_uncen_res, columns = headers).to_csv(path +
                                            '/sub-%s_task-%s_space-fsLR_atlas-Glasser_desc-uncensoredDespiked_parcelations.tsv' %(sub_label, dirM), sep='\t')
        compute_conn(parced_full_uncen_res, path, sub_label, dirM, "uncensored")

        censored_vols.append(sum(censored_vols))
        outlier_vols.append(sum(outlier_vols))
        half_vols.append(sum(half_vols))
        
        pd.DataFrame({'round': [dirM ], 'del vols': censored_vols[-1], 'N of vols needed': half_vols[-1], 'high motion vols': outlier_vols[-1]}).to_csv(path + 
                            '/sub-%s_ses-%s_task-%s_scrubbing_log.tsv' %(sub_label, ses, dirM), sep='\t')
        
        plt.close('all')
        print('done:'+sub_label)
        
        gc.collect()
    except Exception as e:
        #error.append(sub_label)
        logging.error(traceback.format_exc())
    # except ValueError:
    #     print(model.subject_label +'raised value error')
    # except TypeError:
    #     print(model.subject_label +'raised type error')
    # except MemoryError:
    #     print(model.subject_label +'raised memory error')    




In [ ]:
## run parallel
import warnings
import joblib
from joblib import Parallel, delayed
from joblib import parallel_backend

from joblib import Memory
warnings.filterwarnings(action='ignore')

#plt.ioff()
output_dir = data_dir + "derivatives/nilearn_glm/tconnectivity_WM/scrubbed-despiked/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Parallel(n_jobs=40)(delayed(surf_contrast)(iter, subject_label, data_dir, output_dir) for iter, subject_label in enumerate(subject_ID))

